In [2]:
# import libraries

import pandas as pd
import locale

In [3]:
# read in csv and show data frame
listings_path = 'Resources/Listings_Cleaned_Sample.csv'
extract_df = pd.read_csv(listings_path)
prices_df = extract_df[['price','review_scores_value','number_of_reviews']]
prices_df.info()
prices_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                10000 non-null  object 
 1   review_scores_value  7589 non-null   float64
 2   number_of_reviews    10000 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


/var/folders/q0/1ctrg7ln7537xxm0j8j094dm0000gn/T/ipykernel_2552/3812570206.py:3: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  extract_df = pd.read_csv(listings_path)


,price,review_scores_value,number_of_reviews
0,$25.00,NaN,1
1,$115.00,10.0,6
2,$135.00,10.0,1
3,$69.00,9.0,14
4,$130.00,9.0,22


In [4]:
# remove $ symbol from price column and then update data type for further transformation
prices_df['price'] = prices_df['price'].str.replace('$','')
prices_df['price'] = prices_df['price'].replace(',','',regex=True)
prices_df['price'] = prices_df['price'].astype(float)
prices_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price                10000 non-null  float64
 1   review_scores_value  7589 non-null   float64
 2   number_of_reviews    10000 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 234.5 KB


/var/folders/q0/1ctrg7ln7537xxm0j8j094dm0000gn/T/ipykernel_2552/1704648858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['price'] = prices_df['price'].str.replace('$','')
/var/folders/q0/1ctrg7ln7537xxm0j8j094dm0000gn/T/ipykernel_2552/1704648858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['price'] = prices_df['price'].replace(',','',regex=True)
/var/folders/q0/1ctrg7ln7537xxm0j8j094dm0000gn/T/ipykernel_2552/1704648858.py:4: SettingWithCopyWarning: 
A value is trying to be s

In [5]:
# aggregate tracked columns to show the average rating of each price
# sum the number of reviews for each price
clean_prices = prices_df.groupby('price').agg({'review_scores_value':['mean'], 
                                                'number_of_reviews':['sum']}).reset_index()
# join the sum and mean so that the columns are one dimensional
clean_prices.columns = clean_prices.columns.map('_'.join)
clean_prices

,price_,review_scores_value_mean,number_of_reviews_sum
0,10.0,NaN,0
1,12.0,NaN,0
2,15.0,9.250000,272
3,16.0,8.666667,21
4,17.0,10.000000,13
...,...,...,...
489,5000.0,10.000000,6
490,5500.0,NaN,0
491,6600.0,NaN,0
492,7500.0,10.000000,2


In [6]:
clean_prices = clean_prices.rename(columns={'price_':'price',
                             'review_scores_value_mean':'avg_review_per_price'})

In [7]:
# create a new column that identifies if the price is groups cheap, affordable, expensive, or extravagent
# create empty price rating list
affordability = []

# if the value fits the condition then append to list
for i in clean_prices['price']:
    if i <= 50.0:
        affordability.append('Cheap $0-49')
    elif i > 50.0 and i <= 200.0:
        affordability.append('affordable $51-200')
    elif i > 200.0 and i <= 500.0:
        affordability.append('expensive $200-500')
    elif i > 500.0:
        affordability.append('very expensive $500+')
    else:
        affordability.append('NA')
    
len(affordability)


494

In [8]:
# add affordability column to prices data frame
prices_2 = clean_prices.assign(affordability=affordability)
prices_2.head()

,price,avg_review_per_price,number_of_reviews_sum,affordability
0,10.0,NaN,0,Cheap $0-49
1,12.0,NaN,0,Cheap $0-49
2,15.0,9.250000,272,Cheap $0-49
3,16.0,8.666667,21,Cheap $0-49
4,17.0,10.000000,13,Cheap $0-49


In [9]:
# aggregate the affordability and the average rating and total ratings
# this is an average of average with regards to the sum of reviews per grouping NOT the number of reviews
# per price point... bring this up in office hours please
afford_rating_df = prices_2.groupby('affordability').agg({'avg_review_per_price':['mean'], 
                                                'number_of_reviews_sum':['sum']}).reset_index()
afford_rating_df.columns = afford_rating_df.columns.map('_'.join)

afford_rating_df

,affordability_,avg_review_per_price_mean,number_of_reviews_sum_sum
0,Cheap $0-49,9.260476,19591
1,affordable $51-200,9.506454,166320
2,expensive $200-500,9.457092,19648
3,very expensive $500+,9.537335,2426


In [10]:
afford_rating_df = afford_rating_df.rename(columns={'affordability_':'affordability',
                                'avg_price_review_mean':'affordability_reivew_mean',
                                 'number_of_reviews_sum_sum':'total_reviews_per_affordability'
                                })
afford_rating_df

,affordability,avg_review_per_price_mean,total_reviews_per_affordability
0,Cheap $0-49,9.260476,19591
1,affordable $51-200,9.506454,166320
2,expensive $200-500,9.457092,19648
3,very expensive $500+,9.537335,2426


In [11]:
# export to CSV
afford_rating_df.to_csv('data/affordability_vs_rating.csv')


In [12]:
# use locale module to reset prices column to match original dataframe
locale.setlocale(locale.LC_ALL,'')
locale.getlocale()
# locale.currency(10.0, grouping=True)
# for loop to convert prices
formatted_price = []
for i in prices_2['price']:
    i = locale.currency(i, grouping=True )
    formatted_price.append(i)
    
prices_2['price'] = formatted_price
prices_2['price'] = prices_2['price'].astype(object)
prices_2

,price,avg_review_per_price,number_of_reviews_sum,affordability
0,$10.00,NaN,0,Cheap $0-49
1,$12.00,NaN,0,Cheap $0-49
2,$15.00,9.250000,272,Cheap $0-49
3,$16.00,8.666667,21,Cheap $0-49
4,$17.00,10.000000,13,Cheap $0-49
...,...,...,...,...
489,"$5,000.00",10.000000,6,very expensive $500+
490,"$5,500.00",NaN,0,very expensive $500+
491,"$6,600.00",NaN,0,very expensive $500+
492,"$7,500.00",10.000000,2,very expensive $500+


In [13]:
# export to CSV
prices_2.to_csv('data/pricing_vs_reviews.csv')


In [14]:
# merge prices_2 df with original data on price and keep the affordabiltiy column
new_df = pd.merge(extract_df, prices_2, how='left', on='price')
# new_df.info()

In [15]:
new_df = new_df.drop(columns=['avg_review_per_price','number_of_reviews_sum','Unnamed: 0'])
# new_df.info()

In [16]:
# drop columns that we will not use as a group
# new_df = new_df.drop(columns=['notes', 'transit', 'access', 'interaction', 'neighbourhood', 'market', 
#                               'country_code', 'country', 'weekly_price', 'monthly_price', 'instant_bookable', 
#                               'cancellation_policy', 'require_guest_profile_picture', 
#                               'require_guest_phone_verification'])
# new_df

In [17]:
# export to CSV

new_df.to_csv('data/final_listings_cleaned.csv')

In [18]:
listings_df = pd.read_csv('data/final_listings_cleaned.csv')
listings_df[['price','affordability']]

/var/folders/q0/1ctrg7ln7537xxm0j8j094dm0000gn/T/ipykernel_2552/2722040235.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  listings_df = pd.read_csv('data/final_listings_cleaned.csv')


,price,affordability
0,$25.00,Cheap $0-49
1,$115.00,affordable $51-200
2,$135.00,affordable $51-200
3,$69.00,affordable $51-200
4,$130.00,affordable $51-200
...,...,...
9995,$118.00,affordable $51-200
9996,$150.00,affordable $51-200
9997,$49.00,Cheap $0-49
9998,$140.00,affordable $51-200
